In [3]:
# Imports
import json, copy, re
from IPython.display import JSON

In [4]:
# Read and write

def read_json (f):
    f = open(f'./{f}.json')
    data = json.load(f)
    f.close()
    return data

def write_json (f, data):
    jstr = json.dumps(data, indent=4)
    f = open(f'./{f}.json', "w")
    f.write(jstr)
    f.close()

In [5]:
# Edit the data

def assign_line_values (_, color, width):
    _['minzoom'] = 9
    _['maxzoom'] = 20
    _['paint'] = {'line-color': color, 'line-width': width, 'line-opacity': 1}
    
def assign_fill_values (_, color, outline):
    _['minzoom'] = 9
    _['maxzoom'] = 20
    _['paint'] = {'fill-color': color, 'fill-outline-color': outline, 'fill-opacity': 1}
    
def set_visable ( _, vis ):
    if 'layout' not in _:
        _['layout'] = {}    
    _['layout']['visibility'] = 'visible' if vis else 'invisible'
    


In [6]:
# Load data for start

RAW = read_json('layers')
RAW_USED = [row for row in RAW if row['type'] in ['fill', 'line']]


In [7]:
def nest_object (layers) :
    
    result = {}
    
    def add_id ( label, value ) :
        parts = re.split("[_-]", label)
        _ = result
        for p in parts:
            if p not in _:
                _[p] = {}
            _ = _[p]
        _['_'] = value
                
    for l in layers:
        add_id(l['id'],l)
    return result

In [8]:
NESTED = nest_object(RAW_USED)
JSON(NESTED)

<IPython.core.display.JSON object>

In [9]:
def run_for_each (like, fnc):
    target = NESTED
    
    if len(like) > 0:
        parts = like.split('.')
        for p in parts:
            if p in target:
                target = target[p]
            else:
                print('err')
                return
        
    def find_values (obj, apply):
        for i in obj:
            if i == '_':
                apply(obj[i])
            else:
                find_values(obj[i], apply)
                
    find_values(target, fnc)
    

In [10]:
def apply_alteration(like, color, width, outline, show):
    
    def apply (row):
        if row['type'] == 'fill':
            assign_fill_values(row, color, outline)
        if row['type'] == 'line':
            assign_line_values(row, color, width)
        set_visable(row, show)
        
    run_for_each(like, apply)
            

In [11]:
def add_background ( color ):
    NESTED['_'] = {
            "id": "background",
            "type": "background",
            "layout": { "visibility": "visible" },
            "paint": { "background-color": color}
    }

In [12]:
def export ( output ):
    
    _all = []
    
    def save (row):
        _all.append(row)
        
    run_for_each('', save)
    
    
    original = read_json(output)
    original['layers'] = _all
    
    write_json(output, original)
    

In [13]:
JSON(NESTED)

<IPython.core.display.JSON object>

In [28]:
apply_alteration('', '', 0, 0, False)
add_background('#870058')
apply_alteration('road.area', 'white', 5, 1, False)
apply_alteration('road.area', 'white', 5, 1, False)
apply_alteration('rail', '#ffccfd', 5,1,True)
#apply_alteration('ferry', 'white', 3,1,True)
apply_alteration('water', '#FFECCC',3,1,True)
#apply_alteration('landuse', 'white',3,1,True)
#apply_alteration('aeroway', 'white',3,1,True)
#apply_alteration('national', 'white',3,1,True)
#apply_alteration('landuse', '#101010',3,1,True)
apply_alteration('building', '#FFECCC',0,'#D9BBF9',True)

In [29]:
export('../captures/client/test-style')